In [1]:
import os
import numpy as np
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, Add
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import Xception
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.xception import preprocess_input
import pickle

# Load and preprocess images
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(299, 299))
    img = img_to_array(img)
    img = preprocess_input(img)
    return img

# Extract features using pretrained Xception model
def extract_features(model, image_path):
    img = preprocess_image(image_path)
    img = np.expand_dims(img, axis=0)
    features = model.predict(img)
    return features

# Load descriptions from file
def load_descriptions(filename):
    with open(filename, 'r') as f:
        descriptions = f.read()
    return descriptions

# Create a dictionary of image filenames and their corresponding descriptions
def create_descriptions_mapping(descriptions):
    descriptions_mapping = {}
    for line in descriptions.split('\n'):
        tokens = line.split('\t')
        if len(tokens) >= 2:
            image_id, image_desc = tokens[0], tokens[1]
            if image_id not in descriptions_mapping:
                descriptions_mapping[image_id] = []
            descriptions_mapping[image_id].append(image_desc)
    return descriptions_mapping

# Tokenize descriptions and create a vocabulary
def create_tokenizer(descriptions):
    all_desc = []
    for key in descriptions:
        [all_desc.append(desc) for desc in descriptions[key]]
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(all_desc)
    return tokenizer

# Calculate vocabulary size
def get_vocab_size(tokenizer):
    return len(tokenizer.word_index) + 1

# Create sequences of images, input sequences, and output words for an image
def create_sequences(tokenizer, max_length, desc_list, photo):
    X1, X2, y = [], [], []
    for desc in desc_list:
        seq = tokenizer.texts_to_sequences([desc])[0]
        for i in range(1, len(seq)):
            in_seq, out_seq = seq[:i], seq[i]
            in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
            out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
            X1.append(photo)
            X2.append(in_seq)
            y.append(out_seq)
    return np.array(X1), np.array(X2), np.array(y)

# Define the captioning model
def define_model(vocab_size, max_length):
    inputs1 = Input(shape=(2048,))
    fe1 = Dropout(0.5)(inputs1)
    fe2 = Dense(256, activation='relu')(fe1)
    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
    se2 = Dropout(0.5)(se1)
    se3 = LSTM(256)(se2)
    decoder1 = Add()([fe2, se3])
    decoder2 = Dense(256, activation='relu')(decoder1)
    outputs = Dense(vocab_size, activation='softmax')(decoder2)
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

# Define paths and filenames
# Define paths and filenames
image_directory = "D:\\Caption_gen\\Flickr8k_Dataset\\Flicker8k_Dataset"

train_images_file ="D:\\Caption_gen\\Flickr8k_text\\Flickr_8k.trainImages.txt"

descriptions_file = "C:\\Users\\HARISH A K\\Desktop\\python\\Caption_generator\\descriptions.txt"

model_filename = 'model_weights.h5'

# Load Xception model for feature extraction
xception_model = Xception(include_top=False, pooling='avg')

# Load photos
train_imgs = [os.path.basename(line.strip()) for line in open(train_images_file, 'r')]



D:\Anaconda\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Extract features and save them
train_features = {}
for img in tqdm(train_imgs):
    img_path = os.path.join(image_directory, img)
    features = extract_features(xception_model, img_path)
    train_features[img] = features

# Save the features dictionary
with open('features.pkl', 'wb') as f:
    pickle.dump(train_features, f)



  0%|                                                                                         | 0/6000 [00:00<?, ?it/s]

1/1 [==============================] - 3s 3s/step


  0%|                                                                               | 1/6000 [00:02<4:41:30,  2.82s/it]

1/1 [==============================] - 0s 426ms/step


  0%|                                                                               | 2/6000 [00:03<2:28:11,  1.48s/it]

...

100%|█████████████████████████████████████████████████████████████████████████████▉| 5998/6000 [43:41<00:00,  3.08it/s]

1/1 [==============================] - 0s 266ms/step


100%|█████████████████████████████████████████████████████████████████████████████▉| 5999/6000 [43:42<00:00,  3.11it/s]

1/1 [==============================] - 0s 258ms/step


100%|██████████████████████████████████████████████████████████████████████████████| 6000/6000 [43:42<00:00,  2.29it/s]


In [53]:
# Load descriptions
descriptions = load_descriptions(descriptions_file)

# Create descriptions mapping
descriptions_mapping = create_descriptions_mapping(descriptions)

# Create tokenizer
tokenizer = create_tokenizer(descriptions_mapping)

# Determine the maximum sequence length
max_length = max(len(desc.split()) for desc_list in descriptions_mapping.values() for desc in desc_list)

# Calculate vocabulary size
vocab_size = get_vocab_size(tokenizer)

# Define the captioning model
model = define_model(vocab_size, max_length)